In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
from fludarabine import model_helper 
from fludarabine.helper import *
from fludarabine.plotly_helper import *

In [2]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    
#     tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    tree_df = pd.read_csv(f"../Cell_Population/{patient_id}_build_tree_posteriors.tsv", sep = '\t')

    abundance_file = f"../Cell_Population/{patient_id}_cell_population_abundances.tsv"
    mcmc_file = f"../Cell_Population/{patient_id}_cell_population_mcmc_trace.tsv"  
    
    abundance_df = pd.read_csv(abundance_file, sep='\t')
    mcmc_df = pd.read_csv(mcmc_file, sep='\t')  
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count_sample, wbc_all,all_times





In [3]:
end_treatment_df = pd.read_csv("../../../data/gcll/CLL8_end_treatment.csv")

In [4]:
# Input patient information
# Input patient information
patient_id = 'CLL14-1166'
wbc_file = '../../../data/gcll/Copy of CLL14 CLL cell count 07102025 v4_Liang.csv'
treatment_file = '../../../data/gcll/GCLL_treatment.txt'
UMI_start = 1
UMI_end = 4
treatment_start = 0


In [5]:
# Manually verify the sample list order is correct

In [7]:

sample_list = ['CLL14-1166-Tumor-01-pair', 'SM-ODX2C', 'SM-ODX2B', 'SM-ODX2A', 'SM-ODX2D', 'CLL14-1166-Tumor-02-pair'] 

In [7]:
# Output file

In [6]:
outputfile_name = "CLL14_subplot_combined_1166_wbc_updated.html"

Update patient specific information in the below cell 

In [9]:
# Get the treatment duration 

treatment_end = 365

In [10]:
treatment_end

365

In [20]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [21]:
times_sample

[0, 1223, 1388, 1582, 1783, 1874]

In [22]:
wbc_count

[10.92,
 10.92,
 4.4,
 2.21,
 5.74,
 4.62,
 6.54,
 7.46,
 5.83,
 4.96,
 5.64,
 8.77,
 8.2,
 7.63,
 7.84,
 8.03,
 8.03]

In [23]:
CLL_count_sample


[10.5924, 0.352723, 0.120105, 1.057253, 1.924824, 8.03]

In [24]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [25]:
tree_df

,n_iter,likelihood,edges
0,61,"[0.19999999999999998, 0.2, 0.19999999999999998...","1-2,1-3,2-6,3-4,3-5,None-1"
1,34,"[0.25, 0.2, 0.5, 0.5, 0.5, 0.2, 0.333333333333...","1-2,1-3,2-4,3-5,4-6,None-1"
2,30,"[0.3333333333333333, 0.5, 0.5, 0.5, 0.33333333...","1-2,1-3,2-5,3-4,5-6,None-1"
3,21,"[0.5, 0.20000000000000007, 0.2, 0.5, 0.3333333...","1-2,1-3,3-4,3-5,4-6,None-1"
4,20,"[0.3333333333333333, 0.5, 0.5, 0.25, 0.3333333...","1-2,1-3,2-4,2-5,5-6,None-1"
5,17,"[0.24999999999999994, 0.5, 0.5, 0.333333333333...","1-2,1-3,2-5,2-6,3-4,None-1"
6,12,"[0.25, 0.5, 0.3333333333333333, 0.5, 0.3333333...","1-2,1-3,2-4,2-5,4-6,None-1"
7,11,"[0.24999999999999994, 0.25, 0.2, 0.25, 0.33333...","1-2,1-3,2-4,2-6,3-5,None-1"
8,10,"[0.25000000000000006, 0.2, 0.5, 0.5, 0.2500000...","1-2,1-3,3-4,3-5,3-6,None-1"
9,9,"[0.2, 0.25, 0.3333333333333333, 0.5, 0.2, 0.5,...","1-2,1-3,2-5,3-4,4-6,None-1"


In [56]:
 plot_tree_html = plot_tree_plotly(tree_df, 10)

In [27]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [30]:
branch_annotations = {
    1: "MAP2K2",
    5:"ATM"
    
}

In [57]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=10,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

In [34]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [35]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [36]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [37]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [38]:
times_sample

[0, 1223, 1388, 1582, 1783, 1874]

In [39]:
times_sample[extrapolate_start_idx:]

[1223, 1388, 1582, 1783, 1874]

In [40]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [41]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [42]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_15450/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
times_aft_tx

[365, 428, 519, 611, 702, 793, 884, 1067, 1223, 1388, 1582, 1783, 1874]

In [31]:
# Needs to manually check times_sliced_aft

In [44]:
times_sliced_aft

[793, 884, 1067, 1223, 1388, 1582, 1783, 1874]

In [45]:
len(times_sliced_aft)

8

In [47]:
index_samples_model = slice(1,6)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [48]:
y

[None,
 None,
 None,
 array([-7.10682012, -1.06224207, -5.81586732, -8.18792788, -7.06963802,
        -6.06325704]),
 array([-6.94964917, -2.15129209, -8.98175261, -6.38289616, -7.58356162,
        -7.4881173 ]),
 array([-4.7107572 , -0.01709826, -6.18086787, -3.07880393, -5.41273341,
        -4.31999285]),
 array([-3.98699104,  0.52341795, -4.01744361, -2.39997502, -2.64624857,
        -3.21885145]),
 array([-3.84888932,  1.26411095, -3.32491797,  0.82207778, -0.26070171,
         0.33064238])]

In [49]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [50]:
wbc_model

[0.002617, 0.003432, 0.005507, 0.352723, 0.120105, 1.057253, 1.924824, 8.03]

In [51]:
len(wbc_model)

8

In [52]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [53]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [54]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [58]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL14_subplot_combined_1166_wbc_updated.html
